## Practice: Writing Text Files (Advanced)

In these exercises you'll practice writing text files using context managers, write/append modes, and line-by-line processing.


### Problem 1 – Writing a Report File

Create a file called `daily_report.txt` that contains the following lines **exactly**:

```
Report for day 1
Items processed: 10
Errors: 0
```

Use:

* A context manager (`with open(...) as f:`)
* Mode `'w'`
* A **single** call to `write` (i.e. do **not** call `write` three times)


In [1]:
# TODO: Problem 1
# Create daily_report.txt with the exact contents shown above, using a single write call.

report_file = "daily_report.txt"

# your code here



#### Solution 1


In [2]:
report_file = "daily_report.txt"

lines = [
    "Report for day 1",
    "Items processed: 10",
    "Errors: 0"
]

# Join with newline characters and add a final newline at the end (good practice)
contents = "\n".join(lines) + "\n"

with open(report_file, "w", encoding="utf-8") as f:
    f.write(contents)

# Quick check (optional)
with open(report_file, encoding="utf-8") as f:
    print(f.read())


Report for day 1
Items processed: 10
Errors: 0



### Problem 2 – Appending to a Log File

You are maintaining a log file called `events.log`. Each time your script runs, it should append one line to the file.

Append the line:

```text
START
```

to the file using mode `'a'`, and **ensure that each run adds the word `START` on a new line**.

Hints:

* Consider what happens if the file already exists and does **not** end with a newline.
* You may want to **add** a leading newline before `START` only when necessary.


In [3]:
# TODO: Problem 2
# Append the word START to events.log so that each run adds START on its own line.

log_file = "events.log"

# your code here



#### Solution 2


In [4]:
import os

log_file = "events.log"

# Make sure the file exists (so we can inspect the last character)
if not os.path.exists(log_file):
    with open(log_file, "w", encoding="utf-8") as f:
        f.write("")  # create empty file

# Read the last character (if any) to decide if we need a newline
need_newline = True
with open(log_file, "rb") as f:
    try:
        f.seek(-1, os.SEEK_END)
    except OSError:
        # empty file
        need_newline = False
    else:
        last_char = f.read(1)
        # If the last character is already a newline, we don't need another one
        need_newline = last_char != b"\n"

with open(log_file, "a", encoding="utf-8") as f:
    if need_newline:
        f.write("\n")
    f.write("START\n")

# Quick check (optional)
with open(log_file, encoding="utf-8") as f:
    print(f.read())


START
START



### Problem 3 – Filtering a CSV File While Writing

Assume you have the file `DEXUSEU.csv` in the same directory as this notebook.  
The file has the format:

```text
DATE,DEXUSEU
2015-04-03,1.0990
2015-04-06,1.1008
...
```

Create a new file called `DEXUSEU_2016.csv` that contains **only the rows for the year 2016**, including the header.

Requirements:

* Use **line-by-line** reading (do **not** load the entire file into memory).
* Use a context manager for both the source and target file.
* Correctly skip the header in the source file and write it to the target file.


In [5]:
# TODO: Problem 3
source_file = "DEXUSEU.csv"
target_file_2016 = "DEXUSEU_2016.csv"

# your code here



#### Solution 3


In [6]:
source_file = "DEXUSEU.csv"
target_file_2016 = "DEXUSEU_2016.csv"

with open(source_file, encoding="utf-8") as src:
    with open(target_file_2016, "w", encoding="utf-8") as dst:
        # Copy header
        header = next(src)
        dst.write(header)
        
        # Process data lines one at a time
        for line in src:
            date_str, rate = line.strip().split(",")
            year = date_str[:4]
            if year == "2016":
                dst.write(line)

# Quick check (optional – print first few lines)
with open(target_file_2016, encoding="utf-8") as f:
    for _ in range(5):
        print(f.readline().rstrip())


DATE,DEXUSEU
2016-01-01,.
2016-01-04,1.0803
2016-01-05,1.0743
2016-01-06,1.0762


### Problem 4 – Rewriting a File With a Transformation

You are given a text file `names_raw.txt` that contains one name per line, but with inconsistent spacing:

```text
   alice
Bob   
   charlie   
```

Write a script that creates a cleaned file `names_clean.txt` with:

* All names stripped of leading/trailing whitespace
* First letter uppercase, the rest lowercase (i.e. `Alice`, `Bob`, `Charlie`)
* Exactly **one** name per line
* No extra blank lines at the end

Use a read–transform–write loop that processes the file line by line.


In [7]:
# TODO: Problem 4
source_names = "names_raw.txt"
clean_names = "names_clean.txt"

# your code here



In [8]:
sample_names = """   alice
Bob   
   charlie   
"""

with open("names_raw.txt", "w", encoding="utf-8") as f:
    f.write(sample_names)

#### Solution 4


In [9]:
source_names = "names_raw.txt"
clean_names = "names_clean.txt"

with open(source_names, encoding="utf-8") as src, \
     open(clean_names, "w", encoding="utf-8") as dst:
    
    first_written = False
    for line in src:
        name = line.strip()
        if not name:
            # skip empty lines
            continue
        cleaned = name.capitalize()
        
        # Avoid a trailing blank line at the end by controlling newlines manually
        if first_written:
            dst.write("\n")
        dst.write(cleaned)
        first_written = True

# Quick check
with open(clean_names, encoding="utf-8") as f:
    print(repr(f.read()))


'Alice\nBob\nCharlie'


### Problem 5 – Safe File Overwrite Using a Temporary File

Sometimes you want to **modify a file in place**, but doing so directly can leave the file corrupted if something goes wrong mid-way.

Write a function `uppercase_file(path)` that:

1. Reads the file at `path` line by line.
2. Writes an **uppercase** version of each line to a temporary file in the **same directory**.
3. When finished successfully, replaces the original file with the temporary file (using `os.replace`).

If an exception is raised during the transformation, the original file should remain unchanged.

Hints:

* Use `os.path.dirname(path)` and `os.path.join(...)` to create the temporary file path.
* A simple name like `path + ".tmp"` is enough here.


In [10]:
# TODO: Problem 5
import os

def uppercase_file(path: str) -> None:
    """Replace the file at `path` with an uppercased version of its contents.

    The operation should be safe: if something goes wrong while writing,
    the original file must remain unchanged.
    """
    # your code here


# Example usage (uncomment after implementing):
# with open("example.txt", "w", encoding="utf-8") as f:
#     f.write("Hello\nworld\n")
#
# uppercase_file("example.txt")
#
# with open("example.txt", encoding="utf-8") as f:
#     print(f.read())


#### Solution 5


In [11]:
import os

def uppercase_file(path: str) -> None:
    """Replace the file at `path` with an uppercased version of its contents.

    The operation is done via a temporary file and os.replace to avoid
    leaving the original file in a partially written state.
    """
    dir_name = os.path.dirname(path) or "."
    tmp_path = path + ".tmp"

    # Write to temporary file first
    with open(path, encoding="utf-8") as src, \
         open(tmp_path, "w", encoding="utf-8") as tmp:
        for line in src:
            tmp.write(line.upper())

    # Atomically replace the original file with the temp file
    os.replace(tmp_path, path)


# Quick demo (optional)
if __name__ == "__main__":
    demo_file = "example_uppercase_demo.txt"
    with open(demo_file, "w", encoding="utf-8") as f:
        f.write("Hello\nWorld\n")

    uppercase_file(demo_file)

    with open(demo_file, encoding="utf-8") as f:
        print(f.read())


HELLO
WORLD

